📊 **Analysis:** Reliable delivery of critical components

**Objective:** To identify the most active suppliers, understand which suppliers dominate in specific manufacturing regions, and pinpoint any gaps in supply to the Asia region

**Business Logic:** The analysis utilizes Aggregate Function, CTEs, Window Function, SubQueries
- **Supply Chain Gap Analysis:** Utilizes Subqueries to isolate suppliers with zero delivery footprint in the Asia region, enabling the procurement team to mitigate regional supply risks.
- **Regional Dominance Tiering:** Employs CTEs and Window Functions (ROW_NUMBER) to identify the primary supplier for each manufacturing territory, streamlining vendor management.
- **Volume Performance Tracking:** Leverages Aggregate Functions to rank suppliers by component volume, pinpointing the top 5 strategic partners during peak periods.

As a Data Analyst on the Supply Chain Procurement Team, you are tasked with assessing supplier performance to ensure reliable delivery of critical components. Your goal is to identify the most active suppliers, understand which suppliers dominate in specific manufacturing regions, and pinpoint any gaps in supply to the Asia region. By leveraging data, you will help optimize vendor selection strategies and mitigate potential supply chain risks.

Company:
Apple
◆
Difficulty:
Medium
◆
[Question Link](https://www.interviewmaster.ai/question/supplier-performance-metrics)

**Tables**

**supplier_deliveries** (supplier_id, delivery_date, component_count, manufacturing_region)

**suppliers** (supplier_id, supplier_name)

**Question 1 of 3**

We need to know who our most active suppliers are. Identify the top 5 suppliers based on the total volume of components delivered in October 2024.

In [ ]:
# Identify top 5 suppliers by component volume for October 2024.
SELECT supplier_name,
  SUM(component_count) AS total_volume_of_component_delivered
FROM supplier_deliveries
JOIN suppliers ON supplier_deliveries.supplier_id = suppliers.supplier_id
WHERE delivery_date BETWEEN '2024-10-01' AND '2024-10-31'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

**Question 2 of 3**

For each region, find the supplier ID that delivered the highest number of components in November 2024. This will help us understand which supplier is handling the most volume per market.

In [ ]:
# Determine the leading supplier per manufacturing region based on November 2024 throughput.

WITH highest_number_of_component_each_region AS(
  SELECT supplier_deliveries.supplier_id AS supplier_id,
    manufacturing_region,
    SUM(component_count) AS total_number_of_components
  FROM supplier_deliveries
  JOIN suppliers ON supplier_deliveries.supplier_id = suppliers.supplier_id
  WHERE delivery_date BETWEEN '2024-11-01' AND '2024-11-30'
  GROUP BY 1, 2
),
ranking AS (
  SELECT supplier_id,
    total_number_of_components,
    manufacturing_region,
    ROW_NUMBER() OVER(PARTITION BY manufacturing_region ORDER BY total_number_of_components DESC) AS tier_rank
  FROM highest_number_of_component_each_region
)
SELECT *
FROM ranking
WHERE tier_rank = 1
ORDER BY 2 DESC;

**Question 3 of 3**

We need to identify potential gaps in our supply chain for Asia. List all suppliers by name who have not delivered any components to the 'Asia' manufacturing region in December 2024.

In [ ]:
# Audit suppliers with no delivery presence in the Asia region for December 2024.
SELECT DISTINCT s.supplier_name
FROM suppliers s
WHERE s.supplier_id NOT IN (
  SELECT sd.supplier_id
  FROM supplier_deliveries sd
  WHERE sd.delivery_date BETWEEN '2024-12-01' AND '2024-12-31'
    AND sd.manufacturing_region = 'Asia'
);